In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import torch
import torch_geometric as pyg
import networkx as nx
from src.MinAggGNN import MinAggGNN

seed = 0
torch.manual_seed(seed)
rng = np.random.default_rng(seed)

device = torch.device('cuda')
K = 2
m = 2

In [ ]:
import itertools
from data.data_generation import *

P_1_0 = path_graph(1, [1])
P_2_1 = path_graph(2, [1,0], m=m)
G_scale_K = []
for a, b in itertools.product(range(2*K+1),range(2*(K+1))):
    for k in range(1,K):
        A = torch.zeros(K)
        A[0] = a
        A[k] = b
        G_scale_K.append(path_graph(K, A, m=m))
H_K = H_graph(K)
extra_paths_bf_0 = [
    path_graph(2, None, m=m, rng = rng),
    path_graph(2, None, m=m, rng = rng),
    path_graph(2, None, m=m, rng = rng)
]
extra_paths_bf_2 = [
    path_graph(4, None, m=m, rng = rng),
    path_graph(4, None, m=m, rng = rng),
    path_graph(4, None, m=m, rng = rng),
    path_graph(4, None, m=m, rng = rng),
    path_graph(4, None, m=m, rng = rng)
]
for p in extra_paths_bf_2:
    p.x = p.y.reshape(-1,1)

In [3]:
train = G_scale_K
train.append(P_1_0)
train.append(P_2_1)
train.append(H_K)
train = train + extra_paths_bf_0 + extra_paths_bf_2

In [4]:
from tqdm import tqdm
test = []

pbar = tqdm(range(0,200,4))
for i in pbar:
    size = int(torch.randint(5,200,()))
    
    test.append(cycle_test_graph(3, K, rng = rng))
    test.append(cycle_test_graph(4, K, rng = rng))
    test.append(complete_test_graph(size, K, rng = rng))
    test.append(ER_test_graph(size, 0.5, K, rng = rng))

100%|██████████| 50/50 [00:16<00:00,  2.99it/s]


In [5]:
torch.save(train, 'data/training_data.pt')
torch.save(test, 'data/test_data.pt')